
# TorchCodec 学习与速查

> 汇总 TorchCodec 的核心功能、安装指南、解码/采样示例与 API 参考，方便在 PyTorch 视频/音频训练场景中快速查阅。


## Welcome 欢迎来到 TorchCodec 文档
- **TorchCodec 是什么？**
  - Python 库，用于在 CPU 与 CUDA GPU 上将视频/音频解码为 PyTorch 张量。
  - 支持音频编码（视频编码即将上线），完美融入 PyTorch 生态。
  - 面向视频/音频机器学习任务，负责“把媒体文件变成张量数据”。
- **设计理念 / Design Goals**：
  1. Pythonic API，贴合 Python / PyTorch 约定。
  2. 基于系统 FFmpeg 进行编解码，屏蔽 FFmpeg 使用复杂度，保证性能与正确性。
  3. 直接返回 PyTorch 张量，可无缝衔接 transforms、datasets 与训练循环。


## Installation 安装说明

参考 <https://github.com/pytorch/torchcodec#installing-torchcodec> 获取最新指引。


### CPU-only 安装流程
1. **安装 PyTorch**：按官方说明获取与系统匹配的稳定版本。
2. **安装 FFmpeg**：TorchCodec 支持 FFmpeg 主版本 4-7（全平台），在 macOS / Linux 额外支持 8。若需更新版，可使用 conda：
   ```bash
   conda install "ffmpeg<8"
   # 或
   conda install "ffmpeg<8" -c conda-forge
   ```
3. **安装 TorchCodec**：
   ```bash
   pip install torchcodec
   ```
4. **版本兼容表**：
   | torchcodec | torch | Python |
   | --- | --- | --- |
   | main / nightly | main / nightly | >=3.10, <=3.13 |
   | 0.8 | 2.9 | >=3.10, <=3.13 |
   | 0.7 / 0.6 | 2.8 | >=3.9, <=3.13 |
   | 0.5 / 0.4 / 0.3 | 2.7 | >=3.9, <=3.13 |
   | 0.2 | 2.6 | >=3.9, <=3.13 |
   | 0.1 | 2.5 | >=3.9, <=3.12 |
   | 0.0.3 | 2.4 | >=3.8, <=3.12 |


### CUDA 支持安装流程
1. **GPU 条件**：需具备 NVDEC 硬件并支持目标视频格式（参阅 Nvidia GPU support matrix）。
2. **FFmpeg + NVDEC**：安装带 NVDEC 的 FFmpeg（版本 4-7 全平台可用，Linux 额外支持 8）；确认命令：
   ```bash
   ffmpeg -decoders | grep -i nvidia
   ```
3. **CUDA 工具链**：安装匹配的 PyTorch（含 libnpp、libnvrtc）。
4. **安装 TorchCodec**（pip）：
   ```bash
   # 以 CUDA Toolkit 12.6 为例
   pip install torchcodec --index-url=https://download.pytorch.org/whl/cu126
   ```
   未指定 `--index-url` 将安装 CPU 版本。
5. **Windows（实验性）**：建议使用 conda 获取 PyTorch 与 TorchCodec：
   ```bash
   conda install -c conda-forge "torchcodec=*=*cuda*"
   ```

In [ ]:

# 快速验证安装与 CUDA 支持
try:
    import torchcodec
    info = {'TorchCodec version': torchcodec.__version__}
    try:
        import torch
        info['PyTorch version'] = torch.__version__
        info['CUDA available'] = torch.cuda.is_available()
    except ImportError:
        info['PyTorch version'] = '未安装 torch'
    for k, v in info.items():
        print(f"{k}: {v}")
except ImportError as err:
    print('TorchCodec 未安装 ->', err)
    print('安装示例: pip install torchcodec')


## Decoding 解码
> 参阅 `generated_examples/decoding/*.html` 获取逐步讲解。
- **快速入门**：`basic_example` 展示基础视频解码流程。
- **音频解码**：`audio_decoding` 示例介绍如何读取音频流。
- **GPU 解码**：`basic_cuda_example` 展示 CUDA/NVDEC 的使用方式。
- **流式解码**：`file_like` 教程讲解从云端/文件对象高效拉流。
- **并行解码**：`parallel_decoding` 展示多进程/多线程解码技术。
- **片段采样**：`sampling` 教程涉及常规与随机 Clip 抽取。


### TorchCodec 快速体验 · Getting Started with TorchCodec
```python
from torchcodec.decoders import VideoDecoder

decoder = VideoDecoder("path/to/video.mp4", device="cpu")
meta = decoder.metadata
print(meta)
frame0 = decoder[0]  # [3, H, W] uint8
clip = decoder[0:-1:20]  # [N, 3, H, W]
```


### 音频解码 · Audio Decoding
```python
from torchcodec.decoders import AudioDecoder

audio = AudioDecoder("path/to/audio.flac")
print(audio.metadata)
samples = audio[0:16000]  # [channels, num_samples]
```


### GPU 解码 · GPU Decoding
```python
from torchcodec.decoders import VideoDecoder

decoder = VideoDecoder("path/to/video.mp4", device="cuda")
frames = decoder[0:10]  # 张量位于 GPU，可直接送入后续模型
```


### Clip Sampling 片段采样
```python
from torchcodec.samplers import clips_at_regular_timestamps

clips = clips_at_regular_timestamps(
    decoder,
    seconds_between_clip_starts=1.5,
    num_frames_per_clip=4,
    seconds_between_frames=0.1,
)
print(clips.data.shape, clips.pts_seconds.shape)
```


### 试用示例视频 · Generate Sample Video via FFmpeg
```bash
fontfile=/usr/share/fonts/dejavu-sans-mono-fonts/DejaVuSansMono-Bold.ttf
output_video_file=/tmp/output_video.mp4

ffmpeg -f lavfi -i     color=size=640x400:duration=10:rate=25:color=blue     -vf "drawtext=fontfile=${fontfile}:fontsize=30:fontcolor=white:x=(w-text_w)/2:y=(h-text_h)/2:text='Frame %{frame_num}'"     ${output_video_file}
```
然后即可使用 TorchCodec 对生成视频进行加载与采样。


## Using TorchCodec 功能速览
- **VideoDecoder**：读取视频流，支持索引、时间戳检索、批量获取。
- **AudioDecoder**：读取音频流，返回波形张量。
- **Sampler 系列**：常规/随机索引、时间戳、clip 采样。
- **Frame / FrameBatch / AudioSamples**：封装数据与元信息（PTS、duration 等）。


## Benchmark Results 基准测试结果
- 测试平台：22 核 CPU + Nvidia A100（5 个 NVDEC 解码器）。
- 案例：
  - Mandelbrot 视频 (1280x720@60fps, 120s, libx264)。
  - NASA 宣传片 (960x540@29.7fps, 206s, libx264)。
- 绝大部分解码器使用 FFmpeg 6.1.2，TorchVision 使用 4.2.2。
- TorchCodec 中 “approx” 代表使用近似模式搜索，兼顾速度与准确度。
- 详细数据可参考官方 benchmark 图表。


## Interactive Examples 交互式示例
- **Decoding**：
  - `Decoding a video with VideoDecoder`
  - `Decoding audio streams with AudioDecoder`
  - `Accelerated video decoding on GPUs with CUDA and NVDEC`
  - `Streaming data through file-like support`
  - `Exact vs Approximate seek mode`
  - `How to sample video clips`
  - `Parallel video decoding: multi-processing and multi-threading`
  - `Decoding with custom frame mappings`
- **Encoding**：
  - `Encoding audio samples with AudioEncoder`
> 示例链接位于官方文档，可按需查阅并运行。


## API Reference API 参考


### `torchcodec`
- **Frame**：单个视频帧及元数据。
- **FrameBatch**：帧批次（含张量、PTS、duration）。
- **AudioSamples**：音频样本与元信息。


### `torchcodec.decoders`
- **VideoDecoder**：单流视频解码器。
- **AudioDecoder**：单流音频解码器。
- **set_cuda_backend**：上下文管理器，设置 VideoDecoder 的 CUDA 后端。
- **VideoStreamMetadata / AudioStreamMetadata**：流级元信息。
> 视频/音频解码教程见官方文档：`Decoding a video with VideoDecoder`、`Decoding audio streams with AudioDecoder`。


### `torchcodec.encoders`
- **AudioEncoder**：音频编码器（视频编码即将上线）。
> 教程：`Encoding audio samples with AudioEncoder`。


### `torchcodec.samplers`
- **clips_at_regular_indices**：按等距帧索引采样片段。
- **clips_at_random_indices**：随机帧索引采样。
- **clips_at_regular_timestamps**：按等间隔时间戳采样。
- **clips_at_random_timestamps**：按随机时间戳采样。
> 教程：`How to sample video clips`。